In [2]:
import psycopg2
import random
import string
import time
from tqdm import tqdm

In [4]:
# Database Information
DATABASE_NAME = ""
DATABASE_USERNAME = ""
DATABASE_PASSWORD = ""
DATABASE_HOST = ""
DATABASE_POST = ""

# Establishing the connection
conn = psycopg2.connect(
   database=DATABASE_NAME, user=DATABASE_USERNAME, password=DATABASE_PASSWORD, host=DATABASE_HOST, port=DATABASE_POST
)

# Creating a cursor object using the cursor() method
cursor = conn.cursor()